# Multi-Party Computation and Blockchain

- [Documentation](https://hackmd.io/@ChiHaoLu/C2-report-2023)
- [Repository](https://github.com/ChiHaoLu/Multi-Party-Computation-and-Blockchain)


## Set-Up

In [1]:
!pip install secretshare
!pip install eth_account

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from secretshare import Secret, SecretShare, Share
from eth_account import Account
import random
import json

def printDict(dict):
    print(json.dumps(dict, sort_keys=False, indent=4))

## Preprocessing


In [3]:
# Construct the account object
account = Account.create(extra_entropy='KEYSMASH FJAFJKLDSKF7JKFDJ 1530',)
privateKey = int(account.key.hex(), 16)
print("Private Key: ", privateKey)
secret = Secret(privateKey)

Private Key:  1364004802806672667206594457559421125887912662369987619446686036873159176519


In [4]:
# Construct the transaction object
dynamic_fee_transaction = {
        "gas": 100000,
        "maxFeePerGas": 2000000000,
        "maxPriorityFeePerGas": 2000000000,
        "data": "0x616263646566",
        "nonce": 34,
        "to": "0x09616C3d61b3331fc4109a9E41a8BDB7d9776609",
        "value": "0x5af3107a4000",
        "chainId": 4,
    }
print("Transaction object: ")
printDict(dynamic_fee_transaction)

Transaction object: 
{
    "gas": 100000,
    "maxFeePerGas": 2000000000,
    "maxPriorityFeePerGas": 2000000000,
    "data": "0x616263646566",
    "nonce": 34,
    "to": "0x09616C3d61b3331fc4109a9E41a8BDB7d9776609",
    "value": "0x5af3107a4000",
    "chainId": 4
}


In [5]:
print("\nUse the original key to sign the transaction:")
signed = Account.sign_transaction(dynamic_fee_transaction, account.key.hex())
transaction_hash = signed.hash.hex()
print("Transaction Hash: ", transaction_hash)
print("Signature: ")
print("    - v:", signed.v)
print("    - r:", signed.r)
print("    - s:", signed.s)


Use the original key to sign the transaction:
Transaction Hash:  0x03d7b195bb1b3d1accd4de6ca6d12887d1e360af366e1e08b01918f3831abfd2
Signature: 
    - v: 1
    - r: 115534056649080601028857423666309574914260440785429281453131866961546152916390
    - s: 22651388583251149463669817048932771596499926122556774528040687529184289619208


## Produce the Secret Sharing

In [6]:
# Share the secret(private key)
share_count = 3
threshold = 2
shamir = SecretShare(threshold, share_count, secret=secret)
shares = shamir.split()
shareA = shares[0]
shareB = shares[1]
shareC = shares[2]

print("\nShare A (Stored by writing on the Paper): ", shareA)

print("\nShare B (Stored in the Computer): ", shareB)
print('\t- Please enter your password, which is used to encrypt your share B:')
password = input()
encrypted_share_b = Account.encrypt(shareB.value, password)
pointB = shareB.point
print("\t- Encrypted with the Password by AES:")
printDict(encrypted_share_b)

print("\nShare C (Kept by wallet software supplier): ", shareC)
print("\t- Encrypted with the KEK by AES")
kek = random.randint(0,17898947891374192)
encrypted_share_c = Account.encrypt(shareC.value, kek)
pointC = shareC.point
print("\t- The KEK=", kek, "will be stored in the Wallet Supplier Server")
print("\t- The encrypted_share_c=", encrypted_share_c, "will be stored in the User's iCloud/GoogleDrive")


Share A (Stored by writing on the Paper):  AQCNPOD+s+PW677h7Nr1uPol0CtcjtNAfFRqsioBtCvyMw==

Share B (Stored in the Computer):  AgAXdcInXTAADP6+5YyiLlX4Z8VFfA8lnUqIZV5mGMbx9g==
	- Please enter your password, which is used to encrypt your share B:
1234
	- Encrypted with the Password by RSA:
{
    "address": "a2a152b998200b5dba78b8f2aa80f2f321a6d753",
    "crypto": {
        "cipher": "aes-128-ctr",
        "cipherparams": {
            "iv": "f063028896f0eb5353c79fd9b69967f4"
        },
        "ciphertext": "6185e8b407d807b5f0e013d383fdc11887691032b51bda57f48273eb4fc4ab96",
        "kdf": "scrypt",
        "kdfparams": {
            "dklen": 32,
            "n": 262144,
            "r": 1,
            "p": 8,
            "salt": "edefa3a1f4bd632c454536dabb0a2bdf"
        },
        "mac": "4cdba2a01199d7174207432a513e5ceeacdf734bad07501ccece9448dd73c726"
    },
    "id": "69edff0b-454e-4abb-8f3f-22d299cd29e3",
    "version": 3
}

Share C (Kept by wallet software supplier):  AwChrqNQBn

## Scenario 1 - User's Normal

In [7]:
print("\nUser sign the transaction with Share A & Share B")

# Decrypt Share B
print("\t- Please enter your password to decrypt the Share B: ")
password = input()
hex_decrypted_share_b = int((Account.decrypt(encrypted_share_b, password).hex()), base=16)
decrypted_share_b = Share(point=pointB, value=hex_decrypted_share_b)

# Use decrypted Share A & decrypted Share B to sign the transaction
shamirAB = SecretShare(threshold, share_count, shares=[shareA, decrypted_share_b])
secretAB = shamirAB.combine()
signedAB = Account.sign_transaction(dynamic_fee_transaction, int(secretAB))
transaction_hash_AB = signedAB.hash.hex()
print("Transaction Hash: ", transaction_hash_AB)
print("Signature: ")
print("\t- v:", signedAB.v)
print("\t- r:", signedAB.r)
print("\t- s:", signedAB.s)


User sign the transaction with Share A & Share B
	- Please enter your password to decrypt the Share B: 
1234
Transaction Hash:  0x03d7b195bb1b3d1accd4de6ca6d12887d1e360af366e1e08b01918f3831abfd2
Signature: 
	- v: 1
	- r: 115534056649080601028857423666309574914260440785429281453131866961546152916390
	- s: 22651388583251149463669817048932771596499926122556774528040687529184289619208


## Scenario 2 - User's Key Lost

In [8]:
print("\nIf users lost/forget their share A...")

# Decrypt Share B
print("\t- Please enter your password to decrypt the Share B: ")
password = input()
hex_decrypted_share_b = int((Account.decrypt(encrypted_share_b, password).hex()), base=16)
decrypted_share_b = Share(point=pointB, value=hex_decrypted_share_b)

# Decrypt Share C
print("\t- Get the kek from cloud to decrypt the Share C: ")
hex_decrypted_share_c = int((Account.decrypt(encrypted_share_c, kek).hex()), base=16)
decrypted_share_c = Share(point=pointC, value=hex_decrypted_share_c)

# Use decrypted Share B & decrypted Share C to sign the transaction
print("User sign the transaction with Share B & Share C")
shamirBC = SecretShare(threshold, share_count, shares=[decrypted_share_b, decrypted_share_c])
secretBC = shamirBC.combine()
# print("Recover Private Key: ", int(secretBC))
signedBC = Account.sign_transaction(dynamic_fee_transaction, int(secretBC))
transaction_hash_BC = signedBC.hash.hex()
print("Transaction Hash: ", transaction_hash_BC)
print("Signature: ")
print("\t- v:", signedBC.v)
print("\t- r:", signedBC.r)
print("\t- s:", signedBC.s)


If users lost/forget their share A...
	- Please enter your password to decrypt the Share B: 
1234
	- Get the kek from cloud to decrypt the Share C: 
User sign the transaction with Share B & Share C
Transaction Hash:  0x03d7b195bb1b3d1accd4de6ca6d12887d1e360af366e1e08b01918f3831abfd2
Signature: 
	- v: 1
	- r: 115534056649080601028857423666309574914260440785429281453131866961546152916390
	- s: 22651388583251149463669817048932771596499926122556774528040687529184289619208
